Greadsearching su un gradient boosting

In [1]:
"""import os
import time
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from typing import List, Dict 

# --- Importazioni dal tuo file di utility ---
# Assicurati che 'start_utils.py' sia nella stessa cartella
from start_utils import create_features, read_train_data, read_test_data

# --- Importazioni per il Modello Avanzato ---
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
import warnings

def main():
    # Ignora i warning per un output più pulito
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    # === 1. CARICAMENTO DATI ===
    print("--- 1. Caricamento Dati ---")
    COMPETITION_NAME = 'fds-pokemon-battles-prediction-2025'
    # Modifica il percorso se 'input' non è nella stessa cartella di 'start.py'
    DATA_PATH = 'input' 
    
    train_file_path = os.path.join(DATA_PATH, COMPETITION_NAME, 'train.jsonl')
    test_file_path = os.path.join(DATA_PATH, COMPETITION_NAME, 'test.jsonl')

    try:
        train_data = read_train_data(train_file_path)
        test_data = read_test_data(test_file_path)
        print(f"Dati di training caricati: {len(train_data)} battaglie.")
        print(f"Dati di test caricati: {len(test_data)} battaglie.")
    except FileNotFoundError:
        print(f"ERRORE: Dati non trovati. Assicurati che la cartella '{DATA_PATH}/{COMPETITION_NAME}' esista.")
        return

    # === 2. CREAZIONE FEATURE ===
    print("\n--- 2. Creazione Feature (Feature Engineering) ---")
    
    print("Processing training data...")
    train_df = create_features(train_data)
    
    print("\nProcessing test data...")
    test_df = create_features(test_data, is_test=True)

    # === 3. SELEZIONE FEATURE (Il set vincente!) ===
    print("\n--- 3. Selezione Feature (Pre-Training) ---")

    # Questa è la lista di feature che ha dato i risultati migliori (96.31%)
    # Derivata dalla nostra analisi di importanza precedente.
    TOP_FEATURES = [
        'status_change_diff',
        'diff_final_schieramento',
        'diff_final_hp',
        'p1_bad_status_advantage',
        'p1_pct_final_hp',
        'p2_n_pokemon_use',
        'p2_hp_std',
        'diff_hp',
        'p1_hp_std',
        'hp_delta_std'
        # Aggiungi qui altre feature che erano nel tuo 'TOP_FEATURES' se diverse
    ]
    
    # Aggiungiamo le nuove feature di danno che abbiamo integrato
    TOP_FEATURES.extend(['diff_move_damage_mean', 'diff_move_damage_max'])
    
    # Rimuoviamo duplicati se ce ne sono
    TOP_FEATURES = sorted(list(set(TOP_FEATURES))) 
    
    print(f"Utilizzo di {len(TOP_FEATURES)} feature selezionate.")

    TARGET = 'player_won'
    
    # Creiamo i set di dati X/y usando SOLO le feature migliori
    X = train_df[TOP_FEATURES]
    y = train_df[TARGET]
    
    # Prepariamo il set di test
    X_test = test_df[TOP_FEATURES]

    # === 4. OTTIMIZZAZIONE MODELLO (Grid Search) ===
    print("\n--- 4. Ottimizzazione Modello (Grid Search) ---")
    start_time = time.time()

    # Definiamo il modello base
    gb_model = GradientBoostingClassifier(random_state=42)

    # Definiamo la griglia che ha prodotto i risultati migliori
    param_grid = {
        'n_estimators': [300, 500],
        'learning_rate': [0.05, 0.1],
        'max_depth': [5, 7]
    }
    # (Ho ristretto la griglia ai valori vicini a quelli vincenti per velocizzare)

    # Definiamo la Cross-Validation
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Inizializza GridSearch
    grid_search = GridSearchCV(
        estimator=gb_model, 
        param_grid=param_grid, 
        scoring='accuracy', 
        cv=kf, 
        verbose=2, # Mostra i progressi
        n_jobs=-1  # Usa tutti i processori
    )
    
    grid_search.fit(X, y)

    print("\n--- 📊 Risultati Ottimizzazione ---")
    print(f"Migliore Accuratezza in Cross-Validation: {grid_search.best_score_:.4f}")
    print(f"Migliori Iperparametri Trovati: {grid_search.best_params_}")
    
    end_time = time.time()
    print(f"Tempo di ottimizzazione: {(end_time - start_time) / 60:.2f} minuti")

    # === 5. CREAZIONE SUBMISSION ===
    print("\n--- 5. Creazione File di Submission ---")
    
    best_model = grid_search.best_estimator_
    
    predictions = best_model.predict(X_test)

    submission_df = pd.DataFrame({
        'battle_id': test_df['battle_id'],
        TARGET: predictions
    })

    submission_df.to_csv('submission1.csv', index=False)
    print("File 'submission1.csv' creato con successo!")

# --- Esecuzione dello script ---
if __name__ == "__main__":
    main()"""

'import os\nimport time\nimport pandas as pd\nimport numpy as np\nimport json\nfrom tqdm import tqdm\nfrom typing import List, Dict \n\n# --- Importazioni dal tuo file di utility ---\n# Assicurati che \'start_utils.py\' sia nella stessa cartella\nfrom start_utils import create_features, read_train_data, read_test_data\n\n# --- Importazioni per il Modello Avanzato ---\nfrom sklearn.ensemble import GradientBoostingClassifier\nfrom sklearn.model_selection import StratifiedKFold, GridSearchCV\nfrom sklearn.metrics import accuracy_score\nimport warnings\n\ndef main():\n    # Ignora i warning per un output più pulito\n    warnings.simplefilter(action=\'ignore\', category=FutureWarning)\n\n    # === 1. CARICAMENTO DATI ===\n    print("--- 1. Caricamento Dati ---")\n    COMPETITION_NAME = \'fds-pokemon-battles-prediction-2025\'\n    # Modifica il percorso se \'input\' non è nella stessa cartella di \'start.py\'\n    DATA_PATH = \'input\' \n\n    train_file_path = os.path.join(DATA_PATH, COMPET

uso modelli random forest e ada boosting e poi li faccio votare tra loro per capire quale è l'aproccio migliore

In [ ]:
import os
import time
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from typing import List, Dict 

# --- Importazioni dal tuo file di utility ---
from start_utils import build_pokedex, create_features, read_train_data, read_test_data

# --- Importazioni per i Nuovi Modelli ---
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score
import warnings

def main():
    # Ignora i warning per un output più pulito
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    # === 1. CARICAMENTO DATI ===
    print("--- 1. Caricamento Dati ---")
    COMPETITION_NAME = 'fds-pokemon-battles-prediction-2025'
    DATA_PATH = 'input' 
    
    train_file_path = os.path.join(DATA_PATH, COMPETITION_NAME, 'train.jsonl')
    test_file_path = os.path.join(DATA_PATH, COMPETITION_NAME, 'test.jsonl')

    try:
        train_data = read_train_data(train_file_path)
        test_data = read_test_data(test_file_path)
        print(f"Dati di training caricati: {len(train_data)} battaglie.")
        print(f"Dati di test caricati: {len(test_data)} battaglie.")
    except FileNotFoundError:
        print(f"ERRORE: Dati non trovati. Assicurati che la cartella '{DATA_PATH}/{COMPETITION_NAME}' esista.")
        return
    
    # --- 2. COSTRUZIONE POKEDEX  ---
    # Combiniamo i dati per trovare tutti i Pokémon
    all_data = train_data + test_data
    # CHIAMIAMO LA FUNZIONE per creare il dizionario
    POKEDEX = build_pokedex(all_data)

    # === 2. CREAZIONE FEATURE ===
    print("\n--- 2. Creazione Feature (Feature Engineering) ---")
    
    print("Processing training data...")
    train_df = create_features(train_data,POKEDEX)
    
    print("\nProcessing test data...")
    test_df = create_features(test_data,POKEDEX, is_test=True)

    # === 3. SELEZIONE FEATURE ===
    print("\n--- 3. Selezione Feature (Pre-Training) ---")

    # Usiamo le feature che sappiamo essere forti
    # (Sentiti libero di usare la tua lista 'TOP_FEATURES' se l'hai salvata)
    features_da_usare = [col for col in train_df.columns if col not in ['battle_id', 'player_won']]
    
    print(f"Utilizzo di {len(features_da_usare)} feature totali.")

    TARGET = 'player_won'
    
    X = train_df[features_da_usare]
    y = train_df[TARGET]
    X_test = test_df[features_da_usare]

    # === 4. DEFINIZIONE MODELLI (RF + AdaBoost) ===
    print("\n--- 4. Definizione Modelli Non Lineari ---")
    start_time = time.time()

    # Modello 1: RandomForestClassifier
    # (Un buon punto di partenza: 300 alberi, profondità massima 7)
    clf_rf = RandomForestClassifier(
        n_estimators=300, 
        max_depth=7, 
        random_state=42, 
        n_jobs=-1
    )

    # Modello 2: AdaBoostClassifier
    # (Un buon punto di partenza: 500 stimatori, learning rate 0.1)
    clf_ada = AdaBoostClassifier(
        n_estimators=500, 
        learning_rate=0.1, 
        random_state=42
    )

    # Modello 3: VotingClassifier
    # Combina i due modelli. 'voting='soft'' fa la media delle probabilità (di solito è meglio).
    print("Combinazione dei modelli in un VotingClassifier...")
    voting_clf = VotingClassifier(
        estimators=[
            ('rf', clf_rf),  # (nome, modello)
            ('ada', clf_ada) # (nome, modello)
        ],
        voting='soft' # 'soft' fa la media delle probabilità, 'hard' fa votare 0 o 1
    )
    

    # === 5. VALIDAZIONE VELOCE (Opzionale) ===
    # Controlliamo rapidamente le prestazioni prima del training finale
    
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    print("\nValidazione del VotingClassifier (potrebbe richiedere un minuto)...")
    voting_scores = cross_val_score(voting_clf, X, y, cv=kf, scoring='accuracy', n_jobs=-1)
    print(f"Accuratezza media CV del VotingClassifier: {np.mean(voting_scores):.4f}")

    # === 6. TRAINING FINALE E SUBMISSION ===
    print("\n--- 6. Training Modello Finale e Submission ---")
    
    print("Addestramento del VotingClassifier su tutti i dati...")
    voting_clf.fit(X, y) # Addestra il modello combinato
    
    print("Creazione delle predizioni...")
    predictions = voting_clf.predict(X_test)
    
    submission_df = pd.DataFrame({
        'battle_id': test_df['battle_id'],
        TARGET: predictions
    })

    submission_df.to_csv('submission2.csv', index=False)
    
    end_time = time.time()
    print(f"\nFile 'submission2.csv' creato con successo!")
    print(f"Tempo totale esecuzione: {(end_time - start_time) / 60:.2f} minuti")

# --- Esecuzione dello script ---
if __name__ == "__main__":
    main()

--- 1. Caricamento Dati ---
Dati di training caricati: 10000 battaglie.
Dati di test caricati: 5000 battaglie.

--- 2. Creazione Feature (Feature Engineering) ---
Processing training data...


TypeError: create_features() missing 1 required positional argument: 'POKEDEX'